Experiments in Julia analysis scripts

In [9]:
# import dependencies
using DataFrames
using CSV

In [10]:
# importing a CSV file into Julia
exo_planets = CSV.read("data/PSCompPars_2023.02.18_13.40.14.csv", DataFrame; header=169)

# Selecting a subset of the exoplanets columns:
# COLUMN pl_name:        Planet Name
# COLUMN hostname:       Host Name
# COLUMN pl_letter:      Planet Letter
# COLUMN sy_snum:        Number of Stars
# COLUMN sy_pnum:        Number of Planets
# COLUMN discoverymethod: Discovery Method
# COLUMN disc_year:      Discovery Year
# COLUMN disc_facility:  Discovery Facility
# COLUMN disc_telescope: Discovery Telescope
# COLUMN pl_orbsmax:     Orbit Semi-Major Axis [au]
# COLUMN pl_rade:        Planet Radius [Earth Radius]
# COLUMN pl_dens:        Planet Density [g/cm**3]
# COLUMN pl_eqt:         Equilibrium Temperature [K]
# COLUMN st_teff:        Stellar Effective Temperature [K]
# COLUMN st_rad:         Stellar Radius [Solar Radius]
# COLUMN st_mass:        Stellar Mass [Solar mass]

exo_planets = exo_planets[:, ["pl_name","hostname","pl_letter","sy_snum","sy_pnum",
    "discoverymethod","disc_year","disc_facility","disc_telescope","pl_orbsmax",
    "pl_rade","pl_dens","pl_eqt","st_teff","st_rad","st_mass"]]

Row,pl_name,hostname,pl_letter,sy_snum,sy_pnum,discoverymethod,disc_year,disc_facility,disc_telescope,pl_orbsmax,pl_rade,pl_dens,pl_eqt,st_teff,st_rad,st_mass
,String31,String31,String1,Int64,Int64,String31,Int64,String,String,Float64?,Float64?,Float64?,Int64?,Float64?,Float64?,Float64?
1,11 Com b,11 Com,b,2,1,Radial Velocity,2007,Xinglong Station,2.16 m Telescope,1.29,12.1,19.1,missing,4742.0,19.0,2.7
2,11 UMi b,11 UMi,b,1,1,Radial Velocity,2009,Thueringer Landessternwarte Tautenburg,2.0 m Alfred Jensch Telescope,1.53,12.3,13.8,missing,4213.0,29.79,2.78
3,14 And b,14 And,b,1,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,0.83,12.9,3.9,missing,4813.0,11.0,2.2
4,14 Her b,14 Her,b,1,2,Radial Velocity,2002,W. M. Keck Observatory,10 m Keck I Telescope,2.77307,12.6,7.11,missing,5338.0,0.93,0.91
5,16 Cyg B b,16 Cyg B,b,3,1,Radial Velocity,1996,Multiple Observatories,Multiple Telescopes,1.66,13.5,1.26,missing,5750.0,1.13,1.08
6,17 Sco b,17 Sco,b,1,1,Radial Velocity,2020,Lick Observatory,Coude Auxiliary Telescope,1.45,12.9,3.51,missing,4157.0,25.92,1.22
7,18 Del b,18 Del,b,2,1,Radial Velocity,2008,Okayama Astrophysical Observatory,1.88 m Telescope,2.6,12.5,9.21,missing,4979.0,8.5,2.3
8,1RXS J160929.1-210524 b,1RXS J160929.1-210524,b,1,1,Imaging,2008,Gemini Observatory,8.19 m Gemini North Telescope,330.0,18.647,2.54,1700,4060.0,1.31,0.85
9,24 Boo b,24 Boo,b,1,1,Radial Velocity,2018,Okayama Astrophysical Observatory,1.88 m Telescope,0.19,13.9,0.592,missing,4893.0,10.64,0.99
